In [5]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision import models
import matplotlib.pyplot as plt


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

image_transforms = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms)

listsubtrain = list(range(0, len(train), 500))

listest = list(range(1, len(test), 20))
trainset_1 = torch.utils.data.Subset(train, listsubtrain)
trainset_2 = torch.utils.data.Subset(test, listest)



traindata = torch.utils.data.DataLoader(trainset_1,batch_size=20, shuffle=True, num_workers=2,pin_memory=True)

testdata = torch.utils.data.DataLoader(trainset_2,batch_size=5, shuffle=False, num_workers=2,pin_memory=True)

transform_inception = transforms.Resize((299,299))


model = models.inception_v3()
model.aux_logits = False
if torch.cuda.is_available():
    model.cuda()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay = 0.005, momentum = 0.9)
criterion = nn.CrossEntropyLoss().to(device)
def train_test(epochs):
 list_acc = 0
 for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  model.train()
  for i,data in enumerate(traindata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)

    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    
  list_acc += num_correct/tot
  

 num_correct = 0
 tot = 0
 for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
 return (num_correct)/tot , list_acc/epochs

Device available now: cuda
Files already downloaded and verified


/usr/local/lib/python3.8/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [6]:
list_x = [i+1 for i in range(30)]
list_test = []
list_train = []
for i in list_x:
  a,b = train_test(i)
  print(a,b)
  list_test.append(a)
  list_train.append(b)

0.03 0.0
0.106 0.145
0.124 0.22


KeyboardInterrupt: ignored